# Análise da Correlação de Métricas de Países com sua Emissão de Carbono

## Alunos

- Bruno de Marco Appolonio - RA
- João Vitor Vendemiato Fatoretto - 199944
- Lucas Costa de Oliveira - RA
- Natan Beltrão da Cunha Pevidor Carvalho - 184972

## Introdução

Neste projeto, buscamos analisar a correlação entre diversas métricas e a emissão de carbono anual de um país. As métricas avaliadas incluem tanto fatores diretamente relacionados com a pegada de carbono de um país (como desmatamento) tanto como métricas mais gerais (como liberdade de imprensa), permitindo estabelecer relações diversas.

Como metodologia, aplicamos uma regressão linear sobre os diversos parâmetros, permitindo avaliar a relevância de cada um para o cálculo e criando um modelo de predição considerando mudanças nas métricas avaliadas.

## Métricas Utilizadas

Obtivemos as métricas de diversas fontes. Por isso, nem todos os dados estão disponíveis para todos os países avaliados ou para o mesmo ano. Utilizamos o dados mais recentes disponíveis em cada uma das fontes.

### Our World in Data ([fonte](https://ourworldindata.org/co2-emissions))

- Emissão de CO<sub>2</sub> em milhões de toneladas (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)
- PIB em dólares ajustdo para inflação de 2011 (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)
- População (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)

### ONU ([fonte](http://hdr.undp.org/en/indicators))

- **Índice de gini de distribuição de renda familiar (anos variados)**
- IDH (2019)
- Porcentagem da população residindo em áreas urbanas (2019)
- Porcentagem da variação na área de floresta (diferença entre 1990 e 2016)
- **Porcentagem da energia consumida proveniente de combustíveis fósseis (anos variados)**
- Porcentagem do Rendimento Nacional Bruto derivado da extração de recursos naturais (anos variados)
- Índice de desigualdade de gênero (2019)
- Expectativa de vida no nascimento (2019)
- **Porcentagem da população vivendo abaixo da linha da pobreza (anos variados)**
- **Porcentagem do PIB investido em pesquisa e desenvolvimento (anos variados)**
- **Proporção dos gastos públicos em educação e saúde sobre gastos militares (anos variados)**
- Porcentagem de importações e exportações sobre o PIB (anos variados)
- **Índice de desemprego (2019)**

### Center for Systemic Peace ([fonte](https://www.systemicpeace.org/polityproject.html))

- Polity Score, índice que avalia o nível de democracia de cada país - removemos países com valores especiais para evitar interferência na regressão (2018)

### Repórteres sem Fronteira ([fonte](https://rsf.org/en/ranking))

- Índice de liberdade de imprensa (2021)

**Dados em negrito estão disponíveis para menos de 90% dos países selecionados**

In [1]:
import json
import csv

data = {}

co2_file_path = "owid-co2-data.json"
with open(co2_file_path, 'r', encoding='utf8') as json_file:
    co2_data = json.load(json_file)
    for key in co2_data.keys():
        if 'iso_code' in co2_data[key] and key != 'World':
            for i in range(len(co2_data[key]['data'])):
                if 'co2' in co2_data[key]['data'][i] and 'gdp' in co2_data[key]['data'][i] \
                        and 'population' in co2_data[key]['data'][i]:
                    data[co2_data[key]['iso_code']] = {
                        'name': key,
                        'co2_emissions': co2_data[key]['data'][i]['co2'],
                        'gdp': co2_data[key]['data'][i]['gdp'],
                        'population': co2_data[key]['data'][i]['population']               
                    }

X_fields = ['gdp', 'population']
                   
def get_iso3_from_country_name(country_name):
    iso3 = [key for key, values in data.items() if values['name'] == country_name]
    return iso3[0] if iso3 else None

def add_csv_data(file, country_row, data_row, data_name):
    X_fields.append(data_name)
    with open(file, encoding='utf8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        co2_names = [(data[key]['name'], key) for key in data]
        for line in reader:
            country_key = get_iso3_from_country_name(line[country_row])
            if country_key:
                data[country_key][data_name] = float(line[data_row])

def check_missing_countries(file, country_row):
    with open(file, encoding='utf8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        co2_names = [data_dict[key]['name'] for key in data_dict]
        file_names = [line[country_row] for line in reader]
        return [name for name in co2_names if name not in file_names]

add_csv_data('gini-index.csv', 1, -2, 'gini_index')
add_csv_data('hdi.csv', 1, -2, 'hdi')
add_csv_data('urban-population.csv', 1, -2, 'urban_population')
add_csv_data('forest-area-change.csv', 1, -2, 'forest_area_change')
add_csv_data('fossil-fuel-percentage.csv', 1, -2, 'fossil_fuel_percentage')
add_csv_data('natural-resource-depletion.csv', 1, -2, 'natural_resource_depletion')
add_csv_data('gender-inequality-index.csv', 1, -2, 'gender_inequality_index')
add_csv_data('life-expectancy.csv', 1, -2, 'life_expectancy')
add_csv_data('poverty-line.csv', 1, -2, 'poverty_line')
add_csv_data('research-and-development.csv', 1, -2, 'research_and_development')
add_csv_data('education-health-military-expenses.csv', 1, -2, 'education_health_military_expenses')
add_csv_data('exports-imports.csv', 1, -2, 'exports_imports')
add_csv_data('unemployment.csv', 1, -2, 'unemployment')
add_csv_data('democracy.csv', 4, 10, 'democracy')
add_csv_data('press-freedom.csv', 3, 7, 'press_freedom')

print('Países:', len(data.keys()))

Países: 164


## Pré-processamento dos Dados

Antes de executar a regressão, precisamos realizar algumas alterações nos dados para garantir um melhor resultado.

### Imputação de Dados Faltantes

Devido ao uso de diversas fontes de dados diferentes, muitas métricas não estão disponíveis para todos os países. Porém, o método de regressão linear não é capaz de lidar com valores nulos. Por isso, utilizamos um método simples de imputação de dados, onde valores nulos passam a ter o valor da média aritmética dos valores restantes. Essa atribuição tenta atribuir o valor que tenha o menor impacto possível no treinamento.

### Separação de Dados de Treinamento e Teste

Para validar nosso modelo, reservamos 10% dos países para teste no final e 90% para o treinamento e validações necessárias durante seu desenvolvimento.


### Normalização dos Dados

In [36]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

countries = []
X = []
y = []
for values in data.values():
    countries.append(values['name'])
    y.append(values['co2_emissions'])
    X.append([values[field] if field in values else np.nan for field in X_fields])

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imp.fit_transform(X)
y = np.array(y)

X_train, X_test, y_train, y_test, countries_train, countries_test \
        = train_test_split(X, y, countries, test_size=0.1, random_state=0)


def normalize(array):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(array)
    normalized_array = scaler.transform(array)
    return normalized_array

X_train = normalize(X_train)
X_test = normalize(X_test)

print(X_test)


def normalize2(arrayX, arrayY):
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized_array = scaler.fit_transform(arrayX, arrayY)
    return normalized_array

X_train2 = normalize2(X_train, y_train)
X_test2 = normalize2(X_test, y_test)

print(X_test2)
print(y_test)

print("Training countries:", countries_train)
print("Testing countries:", countries_test)

[[0.36917293 0.72660836 0.36741214 0.70240296 0.40458937 0.491353
  0.65935919 0.15322581 0.49128368 0.78877888 0.         0.19148936
  0.05689655 0.22895126 0.05035971 0.35       0.7250816 ]
 [0.00560934 0.17108977 0.63258786 0.15896488 0.         0.11495422
  0.46676134 0.41129032 0.8177496  0.3630363  1.         0.15965446
  0.27241379 0.06757755 0.4028777  0.8        0.31282953]
 [0.00988311 0.05410742 0.68051118 0.48613678 0.50241546 0.
  0.         0.19354839 0.60063391 0.69966997 0.0455192  0.
  0.32241379 0.17282127 0.48201439 0.8        0.59352247]
 [0.15080971 0.08032409 0.15335463 0.97042514 0.49879227 0.34181078
  0.42158516 0.00806452 0.03169572 0.93069307 0.00426743 0.65957447
  0.33793103 0.21935007 0.33093525 1.         0.        ]
 [0.00847406 0.         0.38019169 0.75231054 0.28502415 0.2543235
  0.7386172  0.         0.47226624 0.71617162 0.00284495 0.04255319
  1.         0.16432792 0.47482014 1.         0.31132312]
 [0.03719922 0.2921071  0.5942492  0.39556377 0.4

In [34]:
#Regressao linear normal
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
#Utilizando a mesma regressão logistica com liblinear pois é o recomendado para datasets pequenos(<10000)
reg = LinearRegression().fit(X_train, y_train)
print(reg.score(X_train, y_train))
print("Intercept")
print(reg.intercept_)
print("Coeficiente")
print(reg.coef_)
y_pred = reg.predict(X_test)
errors = mean_squared_error(y_test, y_pred)
print("Erro pela media quadratica")
print(errors)
errors2 = mean_squared_error(y_test, y_pred, squared=False)
print("Erro pela raiz da media quadratica")
print(errors2)
errors3 = mean_absolute_error(y_test, y_pred)
print("Erro pela media absoluta")
print(errors3)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

0.9229161871308197
Intercept
470.7084045401734
Coeficiente
[5990.96132452 1610.20272832  110.16518836 -575.0793537    30.36629522
   91.71530734  -11.20679058   91.57647607 -501.07935382  117.52875522
 -170.43080087  -82.47229296  111.59043216  147.29517233  130.73301169
 -251.62846543  -33.58023819]
Erro pela media quadratica
6481674.1646972895
Erro pela raiz da media quadratica
2545.91322803769
Erro pela media absoluta
1655.842835988827


,Actual,Predicted
0,281.705,3317.904901
1,1.263,131.577651
2,5.448,122.148071
3,67.112,904.906141
4,4.346,-66.254650
5,16.547,609.004971
6,617.960,4949.082889
7,6.638,85.395842
8,0.297,-184.860495
9,233.516,4187.890444


In [23]:
#Regressao linear com arvore de decisão
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
#Utilizando a mesma regressão logistica com liblinear pois é o recomendado para datasets pequenos(<10000)
reg =  tree.DecisionTreeRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_test)
errors = mean_squared_error(y_test, y_pred)
print("Erro pela media quadratica")
print(errors)
errors2 = mean_squared_error(y_test, y_pred, squared=False)
print("Erro pela raiz da media quadratica")
print(errors2)
errors3 = mean_absolute_error(y_test, y_pred)
print("Erro pela media absoluta")
print(errors3)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

Erro pela media quadratica
3592852.9418445886
Erro pela raiz da media quadratica
1895.4822451937102
Erro pela media absoluta
1084.8944705882354


,Actual,Predicted
0,281.705,800.510
1,1.263,1.996
2,5.448,34.111
3,67.112,399.110
4,4.346,17.152
5,16.547,460.043
6,617.960,5292.268
7,6.638,52.360
8,0.297,2.360
9,233.516,2392.360


In [20]:
#Regressao linear com rede neural
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
#Utilizando a mesma regressão logistica com liblinear pois é o recomendado para datasets pequenos(<10000)
reg =  MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
y_pred = reg.predict(X_test)
errors = mean_squared_error(y_test, y_pred)
print("Erro pela media quadratica")
print(errors)
errors2 = mean_squared_error(y_test, y_pred, squared=False)
print("Erro pela raiz da media quadratica")
print(errors2)
errors3 = mean_absolute_error(y_test, y_pred)
print("Erro pela media absoluta")
print(errors3)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

Erro pela media quadratica
8.664155592777927e+20
Erro pela raiz da media quadratica
29434937731.8484
Erro pela media absoluta
18751052114.770878


,Actual,Predicted
0,281.705,-2.963560e+10
1,1.263,-5.542342e+08
2,5.448,-8.957136e+08
3,67.112,-1.216782e+10
4,4.346,-7.828449e+08
5,16.547,-3.081324e+09
6,617.960,-5.698885e+10
7,6.638,-1.581021e+09
8,0.297,-1.051544e+08
9,233.516,-3.419541e+10


In [15]:
#Regressao linear com k vizinhos
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
#Utilizando a mesma regressão logistica com liblinear pois é o recomendado para datasets pequenos(<10000)
reg =  KNeighborsRegressor(n_neighbors=2).fit(X_train, y_train)
y_pred = reg.predict(X_test)
errors = mean_squared_error(y_test, y_pred)
print("Erro pela media quadratica")
print(errors)
errors2 = mean_squared_error(y_test, y_pred, squared=False)
print("Erro pela raiz da media quadratica")
print(errors2)
errors3 = mean_absolute_error(y_test, y_pred)
print("Erro pela media absoluta")
print(errors3)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

Erro pela media quadratica
11231.66730501471
Erro pela raiz da media quadratica
105.97956078893094
Erro pela media absoluta
67.9235588235294


,Actual,Predicted
0,281.705,191.0290
1,1.263,2.6495
2,5.448,6.6460
3,67.112,123.6725
4,4.346,6.9610
5,16.547,17.2405
6,617.960,420.5925
7,6.638,9.8145
8,0.297,0.9485
9,233.516,307.0890
